In [ ]:
# This program obtains timestamps from our videos marking when asterisks appear on our ATM
# @author: Denisolt Shakhbulatov, Kendall Molas, Tristan Gurtler
from PIL import Image
import sys
import pandas as pd
import os
import cv2
import numpy as np
import time

In [ ]:
###
# Input the file names
# @input startTime - Start video from a certain time-frame 00:00:00.000 (hours,minutes,seconds,milliseconds), user-determined
# @input videoFileName - Input video file name
# @input imageFileSaveArea - Folder where extracted images will be saved
# @input frameFileName - CSV file that will contain timestamp data
# @input contrastVal - contrast at which images will be changed to, determined empirically
# @input brightnessVal - increase brightness of certain videos, determined empirically
# @input cropImage - [x,y,z,w] | [x,y] is the image size, and [z,w] is the coordinate on the image where it will start creating
# the size image
###

startTime = ''
videoFileName = ''
videoFileExtension = '.MP4'
videoToBeProcessed = videoFileName + videoFileExtension
imageFileSaveArea = 'images' + videoFileName
frameFileName = 'frames' + videoFileName + '.csv'
contrastVal = '35'
brightnessVal = '100'
cropImage = ''
rotateImage = ''

In [ ]:
# Creates directory for frames to be stored, gets frames of videos, and stores timestamps in a csv file
# There are different options for processing videos
start_time = time.time()
try:
    os.system("mkdir " + imageFileSaveArea)
    #Standard for processing
    os.system("ffmpeg -ss " + startTime + " -i " + videoToBeProcessed + " -an -vf rotate=" + rotateImage + ",crop=" + cropImage + ",eq=contrast=" + contrastVal + " " + imageFileSaveArea + "/%06d.png")
    
    # Use this if video is too dark when processing
    #os.system("ffmpeg -ss " + startTime + " -i " + videoToBeProcessed + " -an -vf crop=" + cropImage + ",eq=contrast=" + contrastVal + ",brightness=" + brightnessVal + " " + imageFileSaveArea + "/%06d.png")

    # May want to use this for the purpose of finding area of where to get coordinates for cropping the image
    #os.system("ffmpeg -ss " + startTime + " -i " + videoToBeProcessed + " -an -vf crop=" + cropImage + " " + imageFileSaveArea + "/%06d.png")
    
    # Takes the timestamps of the video and exports to a csv file
    os.system("ffprobe -f lavfi -i movie=" + videoToBeProcessed + " -show_frames -show_entries frame=pkt_pts_time -of csv=p=0 > " + frameFileName)
    print 'Successful'
except:
    print 'ERROR'

In [ ]:
##
# This function checks if the image we are looking at is actually a PIN entry screen at all
#
# @input image - the image we want to check for being a PIN Entry screen
# @returns whether or not we are in a PIN Entry screen

def determine_background(image, coord_topleft, coord_bottomright):
    total_pixels = 0
    num_white_pixels = 0

    # range through our box and check for white pixels
    for x in range(coord_topleft[0], coord_bottomright[0] + 1):
        for y in range(coord_topleft[1], coord_bottomright[1] + 1):
            curr = (x,y)
            r, g, b = image.getpixel(curr)
            if ((r,g,b) == (255,255,255)):
                num_white_pixels += 1
            total_pixels += 1
    if ((float(num_white_pixels) / total_pixels) == 1):
        return False
    else:
        return True

In [ ]:
time_stamp = 0
frame = 1
indir = imageFileSaveArea
df = pd.DataFrame()

# Has the asterices appear?
firstAsterisk = False
secondAsterisk = False
thirdAsterisk = False
fourthAsterisk = False

for root, dirs, filenames in os.walk(indir):
    for f in sorted(filenames):
        
        # Read Image
        image = cv2.imread(imageFileSaveArea +'/'+f)
        
        # Lowest r,g,b values it will look for used in shapeMask
        lower = np.array([0, 0, 0])
        # Highest r,g,b values it will look for used in shapeMask
        upper = np.array([10,10,10])
        
        # Preprocessing - Bilaterial Filter smoothes images to reduce noise
        bf_image = cv2.bilateralFilter(image, 7, 175, 175)
        
        # Fills what is in that color range of the filtered image
        shapeMask = cv2.inRange(bf_image, lower, upper)

        # Find contours in the image, reads shapeMask image, only looks for the external hierarchy, and uses simple approximation for contouring
        _, contours, _ = cv2.findContours(shapeMask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Is there an amount of astericies present?
        asterisksPresent = 0
        contour_list = []
        for cnts in contours:
            # Approximates the shape
            approx = cv2.approxPolyDP(cnts, 0.01*cv2.arcLength(cnts,True),True)
            # Get the area as well
            area = cv2.contourArea(cnts)
            if ((len(approx) > 0) & (area > 30) or ((len(cnts) > 0) & (area > 5))):
                asterisksPresent = asterisksPresent + 1
                contour_list.append(cnts)
                
        # If the first asterisk is present
        if (asterisksPresent == 1):
            firstAsterisk = True
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the second asterisk present
        elif (asterisksPresent == 2):
            secondAsterisk = True
            firstAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 
        # If the third asterisk is present
        elif (asterisksPresent == 3):
            thirdAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            fourthAsterisk = ''
        # If the fourth asterisk is present
        elif (asterisksPresent == 4):
            fourthAsterisk = True
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
        # If the screen is cleared
        # Don't care about 5th key (?)
        elif (asterisksPresent == 0 or asterisksPresent > 4):
            firstAsterisk = ''
            secondAsterisk = ''
            thirdAsterisk = ''
            fourthAsterisk = '' 

        print 'Asterisks present are: %d in frame: %d' % (asterisksPresent,frame)  
        
        # Checking if background is white
        log = open(os.path.join(root,f),'r')
        bg = Image.open(log)
        pix = bg.load()
        rgb_im = bg.convert('RGB')
        background_topleft = [5,100]
        background_bottomright = [10,115]
        background = determine_background(bg, background_topleft, background_bottomright)
        
        
        # First Asterisk has popped up
        if (background == False):               
            df = df.append(pd.DataFrame({'Background': background, 'First': firstAsterisk, 'Second': secondAsterisk, 'Third': thirdAsterisk, 'Fourth': fourthAsterisk, }, index=[frame]), ignore_index=False)
        else:
            df = df.append(pd.DataFrame({'Background': background, 'First': 'NaN', 'Second': 'NaN', 'Third': 'NaN', 'Fourth': 'NaN', }, index=[frame]), ignore_index=False)
        
        frame = frame + 1


In [ ]:
# Takes the frames csv file and merges it with the dataframe
Time = pd.read_csv(frameFileName, 
                  names = ["Time_stamp"])
result = pd.concat([df, Time], axis=1, join='inner')

outputFileForUser = 'results' + videoFileName + '.csv'


result = result.reindex(columns=['Time_stamp','Background', 'First', 'Second', 'Third', 'Fourth']).to_csv(outputFileForUser, index=True)


In [ ]:
# Load new csv file
outputFileForUser = outputFileForUser + '.csv'
nR = pd.read_csv('../' + outputFileForUser, index_col=0)

In [ ]:
# Booleans to find the first keypress and prevent from printing out every single line of the csv file to output
firstKP = False
secondKP = False
thirdKP = False
fourthKP = False
clr = False

try:
    with open(videoFileName + '.txt', 'w') as textOutput:
        # Iterate through every row in the csv file and check specific columns for True values
        for index, row in nR.iterrows():
            x = index

            ## First Asterisk Occurance
            # Checks first column for True
            if (nR['First'].iloc[index] == True):
               # Check to see if a first keypress was already found
                if (firstKP == False):
                    # Check if it is an actual keypress by seeing if it is constant keypress
                    isTrue = 0
                    for n in range (1,4):
                        if (nR['First'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                if (clr == True):
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                firstKP = True
                    
                # If the user pressed enter before the four PIN sequence is completed
                # Display time at which user presses ENTER 
                if (nR['Background'].iloc[x+1] == True and firstKP == True):
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False

            # If a first keypress was already found, and becomes missing due to a CLEAR, reset boolean
            elif (firstKP == True and pd.isnull(nR['First'].iloc[index]) == True):
                clr = True
                firstKP = False

            ## Second Asterisk Occurance
            # Check second column for True
            if (nR['Second'].iloc[index] == True):
                # Check to see if the second keypress was already found
                if (secondKP == False):
                    # Check if it is an actual keypress by seeing if it is constant keypress
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Second'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                if (clr == True):
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                secondKP = True

                # If the user pressed enter before the four PIN sequence is completed
                if (nR['Background'].iloc[x+1] == True):
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False

            # If a second keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (secondKP == True and pd.isnull(nR['Second'].iloc[index]) == True):
                secondKP = False
                clr = True

            ## Third Asterisk Occurance
            # Check third column for True
            if (nR['Third'].iloc[index] == True):
                # Check to see if the third keypress was already found
                if (thirdKP == False and fourthKP == False):
                    # Check if it is an actual keypress by seeing if it is constant keypress
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Third'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                if (clr == True):    
                                    clr = False
                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                thirdKP = True

                # If the user presses enter before the four PIN sequence is completed.
                if (nR['Background'].iloc[x+1] == True):
                    nR.Time_stamp.iloc[[x+1]]
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[x+1]])
                    textOutput.write('^\n')
                    firstKP = False
                    secondKP = False
                    thirdKP = False

            # If a third keypress was found, and becomes missing due to a CLEAR, reset boolean
            elif (thirdKP == True and pd.isnull(nR['Fourth'].iloc[index]) == True):
                thirdKP = False
                clr = True

            ## Fourth Asterisk Occurance
            # Check fourth column for True
            # Check to see if fourth keypress was found
            if (nR['Fourth'].iloc[index] == True):

                # If a clear was not taken place, display time at which the fourth asterisk had appeared
                if (fourthKP == False):
                    if (clr == True):
                        clr = False
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                    fourthKP = True

            # Check to see if a key was removed
            elif (pd.isnull(nR['Fourth'].iloc[index]) == True and fourthKP == True):
                
                if (nR['Third'].iloc[index] == True):
                    isTrue = 0
                    for n in range(1,4):
                        if (nR['Third'].iloc[index+n] == True):
                            isTrue = isTrue + 1
                            if (isTrue == 3):
                                fourthKP = False
                                clr = True
                            else:
                                firstKP = False
                                secondKP = False
                                thirdKP = False
                                fourthKP = False
                                clr = False

                                textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                                textOutput.write('---------------------------------------\n')
                            
                else:
                    # Once all keys were found and the screen clears, reset all booleans
                    firstKP = False
                    secondKP = False
                    thirdKP = False
                    fourthKP = False
                    clr = False
                    
                    textOutput.write('%s\n' % nR.Time_stamp.iloc[[index]])
                    textOutput.write('---------------------------------------\n')
            
except:
    print 'End of File'
    
# Clean up erroneous text and output to a new text file
with open(videoFileName + '.txt', 'r') as input:
    with open('new' + videoFileName + '.txt','wb') as output: 
        for line in input:
            if (line!="Name: Time_stamp, dtype: float64"+"\n"):
                output.write(line)
print 'Complete'
print "My program took", time.time() - start_time, "to run"